<a href="https://colab.research.google.com/github/Anitayea/Linear_and_Nonlinear_Optimization/blob/main/HW3Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gurobipy

     |████████████████████████████████| 11.5 MB 30.9 MB/s 


In [ ]:
import numpy as np
import gurobipy as grb
import scipy.sparse as spr
import pandas as pd

2.

\begin{align}
  V_{12} = max_{p_i \geq 0} \quad &s\\
  s.t. \quad &s \leq ∑_i pi\prod_{ij} \quad \forall j \in J\\
  &∑_{i\in I} p_i = 1 [t]\\
\end{align}

(a) let $x_i - p_i/s$

\begin{align}
  V_{12} = \quad &max_{x_i \geq 0} \quad s\\
  s.t. \quad &∑_{i \in I} x_i\prod_{ij} \geq 1\\
  &∑_{i\in I} x_i = 1/s\\
\end{align}

therefore:

\begin{align}
  V_{12} = \quad &max_{x_i \geq 0} \quad 1 / ∑_{i \in I} x_i\\
  s.t. \quad &∑_{i \in I} x_i\prod_{ij} \geq 1\\
\end{align}

hence:

\begin{align}
  1/V_{12} = \quad &min_{x_i \geq 0} \quad ∑_{i \in I} x_i\\
  s.t. \quad &∑_{i \in I} x_i\prod_{ij} \geq 1\\
\end{align}


b) compute the dual to(1) and show that it coincides with $min_{q\in \vartriangle_J} max_{q\in \vartriangle_I} p^T \prod q$ 

The dual problem for $V_{12} = V_{21}:$

\begin{align}
  1/V_{21} = \quad &max_{y_j \geq 0} \quad ∑_{j \in J} y_i\\
  s.t. \quad &∑_{j \in J} y_i\prod_{ij} \leq 1\\
\end{align}

We have:

\begin{align}
  1/V_{21} = \quad &max_{y_j \geq 0} \quad ∑_{j \in J} y_i\\
  s.t. \quad &∑_{j \in J} y_i\prod_{ij} \leq 1\\
\end{align}

b) compute the dual to(1) and show that it coincides with $min_{q\in \vartriangle_J} max_{q\in \vartriangle_I} p^T \prod q$

#Alternative proof see below

The dual problem for $V_{12} = V_{21}:$

\begin{align}
  1/V_{21} = \quad &max_{y_j \geq 0} \quad ∑_{j \in J} y_i\\
  s.t. \quad &∑_{j \in J} y_i\prod_{ij} \leq 1\\
\end{align}

since both LPs are feasible, by strong duality, they coincide, which means that $V_{21} = V_{12}$

Denote $x^*$ to be the optimal solution to $V_{12}$ and $y^*$ to $V_{21}$, we have:

$p_i^* = x_i^*/ ∑_{i \in I}x_i^*, q_j^* = y_j^*/ ∑_{j \in J}y_j^*$

which gives us

$(p^*)^T \prod q^* = min_{q \in \vartriangle_J} (p^*)^T \prod q = max_{p \in \vartriangle_I} p^T \prod q^*$

which means that there exist a optimal solution $(p^*, q^*)$ that is the equilibrium in mixed strategy.

(c) Deduce value of the game, and eq strategies

In [ ]:
Pi_i_j = np.array([[3, 2, 7, 5], 
                    [10, 8, 3, 1], 
                    [2, 9, 4, 6],
                   [4, 5, 1, 7]])
Pi_i_j

matrixgame_model = grb.Model()
nbi,nbj = Pi_i_j.shape
p_iv = matrixgame_model.addMVar(shape = nbi)
sv = matrixgame_model.addMVar(1, lb = - grb.GRB.INFINITY )
matrixgame_model.setObjective(sv, grb.GRB.MAXIMIZE)
matrixgame_model.addConstr( np.ones( (nbj,1) ) @ sv - Pi_i_j.T @ p_iv   <=0  )
matrixgame_model.addConstr( p_iv.sum()==1 )
matrixgame_model.optimize() 

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 5 rows, 5 columns and 24 nonzeros
Model fingerprint: 0x64aaa09d
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 4 rows, 5 columns, 19 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+01   3.125000e+00   0.000000e+00      0s
       3    4.6258278e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.625827815e+00


In [ ]:
p_i = np.array(matrixgame_model.getAttr('x'))[:-1]
p_i

array([0.48013245, 0.21854305, 0.10264901, 0.1986755 ])

In [ ]:
q_j = np.array(matrixgame_model.getAttr('pi'))[:-1]
q_j

array([0.29139073, 0.0794702 , 0.22350993, 0.40562914])

In [ ]:
np.array(matrixgame_model.getAttr('x'))[-1]

4.625827814569536

In [ ]:
np.array(matrixgame_model.getAttr('pi'))[-1]

4.6258278145695355